In [1]:
#Setup
"""
#TODO: Verify installation
cd ~
module load python3/3.11.0
module load openmpi/4.0.5
python3 -m venv envNew --system-site-package
envNew/bin/pip3 install --no-binary :all: mpi4py
envNew/bin/pip3 install --no-binary :all: mgmetis
envNew/bin/pip3 install --no-binary :all: jupyter


#Run
$jupyter nbconvert --execute --clear-output solver_demo.ipynb
"""

'\n#TODO: Verify installation\ncd ~\nmodule load python3/3.11.0\nmodule load openmpi/4.0.5\npython3 -m venv envNew --system-site-package\nenvNew/bin/pip3 install --no-binary :all: mpi4py\nenvNew/bin/pip3 install --no-binary :all: mgmetis\nenvNew/bin/pip3 install --no-binary :all: jupyter\n\n\n#Run\n$jupyter nbconvert --execute --clear-output solver_demo.ipynb\n'

In [2]:
%%bash

cat <<EOL > setup_env.sh
source ~/env/bin/activate
export PYTHONPATH=~/env/lib/python3.7/site-packages:\$PYTHONPATH
module load python3/3.7.4
module load openmpi/4.0.7

export NSplits=4
export NPrt=12
export NMetisParts=1
export Model=CubeTest
export SolverFile=Elast_StaticAnalysis.py
export WorkDir=~/MatlabLink_4Jul/High_Performance_Computing_MPI/
cd \$WorkDir
EOL


In [3]:
%%bash
source setup_env.sh
mpiexec -np $NMetisParts --map-by numa python3 MetisMeshPart.py $WorkDir


Am 0.00556492805480957
Bm 0.0004589557647705078
Cm 0.0006043910980224609
Dm 0.02531719207763672
N_Parts:  12
Em 0.0048215389251708984


In [4]:
%%bash
source setup_env.sh
mpiexec -np $NSplits --map-by numa python3 Elast_ParDataPrepMPI.py $NPrt $WorkDir 0


N_Workers 4
0 A
0 B0
0 B1
0 C
0 D
0 E
0 F
0 G
0 H
0 I
0 J
0 K


In [5]:
%%bash
source setup_env.sh
mpiexec -np $NPrt --map-by numa python3 $SolverFile $Model $WorkDir 1 0 0




N_Workers 12
Time (sec) taken to read files:  0.04
0 1.0 [0.3 0. ]
100 1.3422058681621622e-05 [0.3 0. ]
Analysis Finished Sucessfully..
TotalTimeStepCount 2


-----------------
Verify Total Time: 0.024558346874226464 0.08506858348846436
Total Time: 0.42784690856933594 ['Mean', 0.42509257793426514] ['Max_i', 0.4277353286743164]




In [6]:
%%bash
source setup_env.sh
mpiexec -np 1 python3 ExportVTK.py $Model $WorkDir 1 "U" "Full"


0
1


In [7]:
%%bash
source setup_env.sh
mpiexec -np 1 python3 ExportMat.py $Model $WorkDir 1 "U"

0
1
